In [1]:
%matplotlib inline


# Comparing randomized search and grid search for hyperparameter estimation


Compare randomized search and grid search for optimizing hyperparameters of a
random forest.
All parameters that influence the learning are searched simultaneously
(except for the number of estimators, which poses a time / quality tradeoff).

The randomized search and the grid search explore exactly the same space of
parameters. The result in parameter settings is quite similar, while the run
time for randomized search is drastically lower.

The performance is slightly worse for the randomized search, though this
is most likely a noise effect and would not carry over to a held-out test set.

Note that in practice, one would not search over this many different parameters
simultaneously using grid search, but pick only the ones deemed most important.



In [5]:
print(__doc__)

import numpy as np
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib
from matplotlib import pyplot as plt
import os
import csv
from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

# get some data

# The data, shuffled and split between train and test sets:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# build a classifier
def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32*3, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32*3, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))        

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy']) 
    return model

clf = KerasClassifier(build_fn=build_model, verbose=1)


# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# specify parameters and distributions to sample from
param_dist = dict(epochs = [50,100,150], batch_size = [32, 64, 128, 256, 512])

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(clf, 
                                   scoring = 'accuracy',
                                   n_jobs = -1,
                                   pre_dispatch=4,
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(x_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

Automatically created module for IPython interactive environment
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Epoch 1/100
Epoch 1/100
33333/33333 [==============================] - 5s 159us/step - loss: 2.1578 - acc: 0.1701
Epoch 2/100
33333/33333 [==============================] - 3s 81us/step - loss: 1.9392 - acc: 0.2611
Epoch 3/100
33333/33333 [==============================] - 3s 80us/step - loss: 1.8322 - acc: 0.3085
Epoch 4/100
33333/33333 [==============================] - 3s 80us/step - loss: 1.7595 - acc: 0.3407
Epoch 5/100
33333/33333 [==============================] - 3s 80us/step - loss: 1.6806 - acc: 0.3785
Epoch 6/100
33333/33333 [==============================] - 3s 80us/step - loss: 1.6119 - acc: 0.4049
Epoch 7/100
33333/33333 [==============================] - 3s 80us/step - loss: 1.5550 - acc: 0.4271
Epoch 8/100
33333/33333 [==============================] - 3s 80us/step - loss: 1.5029 - acc: 0.4474
Epoch 9/100
33333/33333 [================

33333/33333 [==============================] - 3s 80us/step - loss: 0.6922 - acc: 0.7559
Epoch 79/100
33333/33333 [==============================] - 3s 79us/step - loss: 0.6836 - acc: 0.7595
Epoch 80/100
33333/33333 [==============================] - 3s 79us/step - loss: 0.6846 - acc: 0.7578
Epoch 81/100
33333/33333 [==============================] - 3s 80us/step - loss: 0.6738 - acc: 0.7609
Epoch 82/100
33333/33333 [==============================] - 3s 79us/step - loss: 0.6694 - acc: 0.7611
Epoch 83/100
33333/33333 [==============================] - 3s 80us/step - loss: 0.6695 - acc: 0.7630
Epoch 84/100
33333/33333 [==============================] - 3s 80us/step - loss: 0.6655 - acc: 0.7671
Epoch 85/100
33333/33333 [==============================] - 3s 79us/step - loss: 0.6560 - acc: 0.7681
Epoch 86/100
33333/33333 [==============================] - 3s 80us/step - loss: 0.6513 - acc: 0.7714
Epoch 87/100
33333/33333 [==============================] - 3s 79us/step - loss: 0.6489 - acc: 

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/dist-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.5/runpy.py in _run_code(code=<